In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *


In [0]:
df = spark.read.format("parquet")\
    .load("abfss://bronze@etedatabricks01.dfs.core.windows.net/orders")

In [0]:
display(df)
df.count()

In [0]:
df = df.withColumnRenamed("_rescued_data", "rescued_data")
df.printSchema()

In [0]:
df = df.drop("rescued_data")

In [0]:
from pyspark.sql.functions import to_timestamp, col
df = df.withColumn("order_date", to_timestamp(col("order_date")))
df.display()

In [0]:
from pyspark.sql.functions import year
df = df.withColumn("year", year(col("order_date")))
df.display()

In [0]:
df_new_ranked_col = df.withColumn("flag",dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
df_new_ranked_col = df_new_ranked_col.withColumn("rank_flag", rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
df_new_ranked_col.display()

In [0]:
class windows:
    
    
    def dense_rank(self,df):

        df_dense_rank = df.withColumn("flag", dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
        return df_dense_rank
    
    def rank(self,df):

        df_rank = df.withColumn("rank_flag", rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
        return df_rank

    def row_number(self,df):

        df_row_number = df.withColumn("row_flag", row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
        return df_row_number
    

In [0]:
obj = windows()
df_new_ranked_col= obj.row_number(df_new_ranked_col)
display(df_new_ranked_col)

In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@etedatabricks01.dfs.core.windows.net/orders")


In [0]:
 %sql
CREATE TABLE IF NOT EXISTS databricks_catalog.silver.orders_silver 
USING DELTA
LOCATION 'abfss://silver@etedatabricks01.dfs.core.windows.net/orders'

